In [1]:
import json
import pandas as pd
import os
from datetime import datetime
import csv
from collections import OrderedDict

In [2]:
def game_reader(minutes):
    to_csv = []
    to_csv_diffs = []
    path = os.getcwd() + '/Games/'
    for filename in os.listdir(path):
        with open(path + filename, 'r') as json_data:
            try:
                data = json.load(json_data)

                blue_dragon = 0
                red_dragon = 0
                blue_riftherald = 0
                red_riftherald = 0
                blue_ward = 0
                red_ward = 0
                blue_destroyed_ward = 0
                red_destroyed_ward = 0
                blue_top_turret = 0
                blue_mid_turret = 0
                blue_bot_turret = 0
                red_top_turret = 0
                red_mid_turret = 0
                red_bot_turret = 0
                blue_inhibitor = 0
                red_inhibitor = 0
                blue_kills = 0
                blue_assists = 0
                red_kills = 0
                red_assists = 0
                blue_cs = 0
                blue_jg = 0
                blue_xp = 0
                red_cs = 0
                red_jg = 0
                red_xp = 0
                blue_gold = 0
                red_gold = 0
                winner = 0

                save_game = True
                    
                if datetime.fromtimestamp(int(data['matchCreation'])/1000)>=datetime(2016, 1, 28) and datetime.fromtimestamp(int(data['matchCreation'])/1000)>=minutes:
                    # Get match winner
                    if data['teams'][0]['winner']==True:
                        winner = data['teams'][0]['teamId']
                    else:
                        winner = data['teams'][1]['teamId']

                    # Get Team & Role for each Participant
                    participant_data = [] 
                    for participant in data['participants']:
                        participant_data.append({
                                                'participantId': participant['participantId'],
                                                'championId': participant['championId'],
                                                'teamId': participant['teamId'],
                                                'lane': participant['timeline']['lane'],
                                                'role': participant['timeline']['role']
                                                })

                    # Search events through the timeline
                    try:
                        for timeline in data['timeline']['frames']:
                            if timeline['timestamp']<=((minutes+1)*60000-10000):
                                try:

                                    # Get cs and xp at 10 minutes
                                    if timeline['timestamp']>(minutes*60000-10000):
                                        for participants in timeline['participantFrames']:
                                            if [participant for participant in participant_data if participant['participantId'] == int(str(participants))][0]['teamId']==100:
                                                blue_cs += timeline['participantFrames'][str(participants)]['minionsKilled']
                                                blue_jg += timeline['participantFrames'][str(participants)]['jungleMinionsKilled']
                                                blue_xp += timeline['participantFrames'][str(participants)]['xp']
                                                blue_gold += timeline['participantFrames'][str(participants)]['totalGold']
                                            else:
                                                red_cs += timeline['participantFrames'][str(participants)]['minionsKilled']
                                                red_jg += timeline['participantFrames'][str(participants)]['jungleMinionsKilled']
                                                red_xp += timeline['participantFrames'][str(participants)]['xp']
                                                red_gold += timeline['participantFrames'][str(participants)]['totalGold']

                                    for event in timeline['events']:

                                         # Get Dragon & Rift Herald Kills
                                        if event['eventType']=='ELITE_MONSTER_KILL' and event["killerId"]!=0:
                                            if event['monsterType']=='DRAGON':
                                                if [participant for participant in participant_data if participant['participantId'] == event['killerId']][0]['teamId']==100:
                                                    blue_dragon += 1
                                                else:
                                                    red_dragon += 1
                                            elif event['monsterType']=='RIFTHERALD' and event["killerId"]!=0:
                                                if [participant for participant in participant_data if participant['participantId'] == event['killerId']][0]['teamId']==100:
                                                    blue_riftherald += 1
                                                else:
                                                    red_riftherald += 1

                                        # Get Wards Placed
                                        if event['eventType']=='WARD_PLACED' and event["creatorId"]!=0:
                                            if [participant for participant in participant_data if participant['participantId'] == event['creatorId']][0]['teamId']==100:
                                                blue_ward += 1
                                            else:
                                                red_ward += 1

                                        # Get Wards Destroyed
                                        if event['eventType']=='WARD_KILL' and event["killerId"]!=0:
                                            if [participant for participant in participant_data if participant['participantId'] == event['killerId']][0]['teamId']==100:
                                                blue_destroyed_ward += 1
                                            else:
                                                red_destroyed_ward += 1

                                        # Get Towers & Inhibitors Destroyed
                                        if event['eventType']=='BUILDING_KILL' and event["killerId"]!=0:
                                            if event['buildingType']=='TOWER_BUILDING':
                                                if event['laneType']=='TOP_LANE':
                                                    if event['teamId']==100:
                                                        red_top_turret += 1
                                                    else:
                                                        blue_top_turret += 1
                                                elif event['laneType']=='MID_LANE':
                                                    if event['teamId']==100:
                                                        red_mid_turret += 1
                                                    else:
                                                        blue_mid_turret += 1
                                                elif event['laneType']=='BOT_LANE':
                                                    if event['teamId']==100:
                                                        red_bot_turret += 1
                                                    else:
                                                        blue_bot_turret += 1
                                            elif event['buildingType']=='INHIBITOR_BUILDING':
                                                if event['teamId']==100:
                                                    red_inhibitor += 1
                                                else:
                                                    blue_inhibitor += 1    

                                        # Get Kills and Assists
                                        if event['eventType']=='CHAMPION_KILL' and event['killerId']!=0:
                                            if [participant for participant in participant_data if participant['participantId'] == event['killerId']][0]['teamId']==100:
                                                blue_kills += 1
                                                blue_assists += len(event['assistingParticipantIds'])
                                            else:
                                                red_kills += 1
                                                red_assists += len(event['assistingParticipantIds'])                                  




                                except KeyError:
                                    pass
                    except KeyError:
                        save_game = False
                else:
                    save_game = False

                if save_game:
                    # Save all game data in a python dict
                    game_data = OrderedDict({'matchId': data['matchId']})
                    game_data['matchVersion'] = data['matchVersion']
                    game_data['region'] = data['region']
                    game_data['queueType'] = data['queueType']
                    game_data['matchCreation'] = datetime.fromtimestamp(int(data['matchCreation'])/1000)
                    game_data['matchDuration'] = data['matchDuration']/60
                    game_data['winner'] = winner
                    game_data['blueGold'] = blue_gold
                    game_data['redGold'] = red_gold
                    game_data['blueDragons'] = blue_dragon
                    game_data['blueRiftHerald'] = blue_riftherald
                    game_data['redDragons'] = red_dragon
                    game_data['redRiftHerald'] = red_riftherald
                    game_data['blueWards'] = blue_ward
                    game_data['redWards'] = red_ward
                    game_data['blueDestroyedWards'] = blue_destroyed_ward
                    game_data['redDestroyedWards'] = red_destroyed_ward
                    game_data['blueTopTurrets'] = blue_top_turret
                    game_data['blueMidTurrets'] = blue_mid_turret
                    game_data['blueBotTurrets'] = blue_bot_turret
                    game_data['redTopTurrets'] = red_top_turret
                    game_data['redMidTurrets'] = red_mid_turret
                    game_data['redBotTurrets'] = red_bot_turret
                    game_data['blueInhibitor'] = blue_inhibitor
                    game_data['redInhibitor'] = red_inhibitor
                    game_data['blueKills'] = blue_kills
                    game_data['blueAssists'] = blue_assists
                    game_data['redKills'] = red_kills
                    game_data['redAssists'] = red_assists
                    game_data['blueCs'] = blue_cs
                    game_data['blueJg'] = blue_jg
                    game_data['blueXp'] = blue_xp
                    game_data['redCs'] = red_cs
                    game_data['redJg'] = red_jg
                    game_data['redXp'] = red_xp

                    # Append this dict to the list of games to be written in a csv
                    to_csv.append(game_data)

                    game_data = OrderedDict({'matchId': data['matchId']})
                    game_data['matchVersion'] = data['matchVersion']
                    game_data['region'] = data['region']
                    game_data['queueType'] = data['queueType']
                    game_data['matchCreation'] = datetime.fromtimestamp(int(data['matchCreation'])/1000)
                    game_data['matchDuration'] = data['matchDuration']/60
                    game_data['winner'] = winner
                    game_data['Gold'] = blue_gold - red_gold
                    game_data['Dragons'] = blue_dragon - red_dragon
                    game_data['RiftHerald'] = blue_riftherald - red_riftherald
                    game_data['Wards'] = blue_ward - red_ward
                    game_data['DestroyedWards'] = blue_destroyed_ward - red_destroyed_ward
                    game_data['TopTurrets'] = blue_top_turret - red_top_turret
                    game_data['MidTurrets'] = blue_mid_turret - red_mid_turret
                    game_data['BotTurrets'] = blue_bot_turret - red_bot_turret
                    game_data['Inhibitor'] = blue_inhibitor - red_inhibitor
                    game_data['Kills'] = blue_kills - red_kills
                    game_data['Assists'] = blue_assists - red_assists
                    game_data['Cs'] = blue_cs - red_cs
                    game_data['Jg'] = blue_jg - red_jg
                    game_data['Xp'] = blue_xp - red_xp

                    to_csv_diffs.append(game_data)

                json_data.close()
            except ValueError:
                pass

    with open(os.getcwd() + '/gameData_' + str(minutes) +'Minutes.csv', 'w') as csv_file:
        writer = csv.DictWriter(csv_file,fieldnames=to_csv[0],dialect='excel')
        writer.writeheader()
        for game in to_csv:
            writer.writerow(game)
    csv_file.close
    with open(os.getcwd() + '/gameData_Diffs_' + str(minutes) +'Minutes.csv', 'w') as csv_file:
        writer = csv.DictWriter(csv_file,fieldnames=to_csv_diffs[0],dialect='excel')
        writer.writeheader()
        for game in to_csv_diffs:
            writer.writerow(game)
    csv_file.close

In [5]:
game_reader(5)
game_reader(10)
game_reader(15)
game_reader(20)

KeyboardInterrupt: 